## Import packages

In [1]:
from math import pi
import matplotlib.pyplot as plt
from pennylane import numpy as np
import pennylane as qml
from pennylane.optimize import AdamOptimizer
import datetime
now=datetime.datetime.now

### Define the objective function

The target function is $f(x_1,x_2,x_3)=0.5\sin(x_1)\sin(x_2)-0.6\cos(x_2)\sin(x_3)+\cos^2(x_3)$.

In [2]:
coef=1
def my_objective(X1,X2,X3):
    Y=0.5*np.sin(coef*X1)*np.sin(coef*X2)-0.6*np.cos(coef*X2)*np.sin(coef*X3)+np.cos(coef*X3)**2
    return Y

In [3]:
my_objective(0,0,pi/2)

-0.6

## Generate data

In [4]:
number=1000
np.random.seed(0)
X=np.random.uniform(-0.95,0.95,size=(number,3))
print(X.shape)

(1000, 3)


In [5]:
Y=my_objective(X[:,0],X[:,1],X[:,2])
print(Y.shape)
print(Y[0:5])

(1000,)
[0.87395657 0.75647989 0.02453891 0.91118912 0.81851272]


## Set device

In [6]:
num_qubits=4
dev=qml.device('default.qubit', wires=num_qubits)

## Define embedding layer

In [7]:
# define my own embedding layer
def myembedding(x,wires):
    qml.RY(coef*x[0], wires=wires[1])
    qml.RY(coef*x[1], wires=wires[2])
    qml.RY(coef*x[2], wires=wires[3])

## Define the Hamiltonian matrix transformation layer

In [8]:
def Ham():
    obs=[]
    for j in range(num_qubits):
        obs.append(qml.PauliX(j))
        for k in range(j):
            obs.append(qml.PauliZ(j)@qml.PauliZ(k))
    coeffs=np.random.uniform(-1,1,len(obs))*10
    qml.Hamiltonian(coeffs, obs)

## Define ansatze layer

In [9]:
# define ansastz layer
def layer(theta):
    
    # Apply Hamiltonian matrix
    Ham()
    
    # Apply H gate
    qml.Hadamard(0)
    
    # rotations on qubit 1
    qml.RY(theta[0],wires=1)
    
    # rotations on qubit 2
    qml.RY(theta[1],wires=2)
    
    # rotations on qubit 3
    qml.RY(theta[2],wires=3)
    
    # CNOT
    qml.CNOT(wires=[0, 1])
    qml.CNOT(wires=[0, 2])
    qml.CNOT(wires=[0, 3])

In [10]:
@qml.qnode(dev)
def quantum_net(theta,x):
    
    # encode data
    myembedding(x,wires=range(num_qubits))
    
    # parameterized circuit layers
    for v in theta: # (for lool along with the first dimension)
        # print(v)
        # Ham()
        layer(v)
    
    qml.Hadamard(0)
    
    return qml.expval(qml.PauliZ(0)),qml.expval(qml.PauliZ(3))

In [11]:
num_layers = 4
theta = np.random.uniform(0,2*pi, size=(num_layers,num_qubits-1), requires_grad=True)
print(theta.shape)
print(theta)

(4, 3)
[[2.60100307 3.95600847 4.89198921]
 [5.35049542 5.12967237 1.04349312]
 [5.20492506 0.36837724 1.25770949]
 [3.91396401 0.72063439 3.79094474]]


In [12]:
quantum_net(theta,[0,0,0])

tensor([-0.29240526, -0.10825549], requires_grad=True)

In [13]:
print(qml.draw(quantum_net)(theta,[0,0,0]))

0: ──H──────────────────╭●─╭●─╭●──H────────╭●─╭●─╭●──H────────╭●─╭●─╭●──H────────╭●─╭●─╭●──H─┤  <Z>
1: ──RY(0.00)──RY(2.60)─╰X─│──│───RY(5.35)─╰X─│──│───RY(5.20)─╰X─│──│───RY(3.91)─╰X─│──│─────┤     
2: ──RY(0.00)──RY(3.96)────╰X─│───RY(5.13)────╰X─│───RY(0.37)────╰X─│───RY(0.72)────╰X─│─────┤     
3: ──RY(0.00)──RY(4.89)───────╰X──RY(1.04)───────╰X──RY(1.26)───────╰X──RY(3.79)───────╰X────┤  <Z>


## Add classical layer

In [14]:
# add the classical layer
def classical_quantum_net(theta,w,x):
    r1=quantum_net(theta,x)[0]
    r2=quantum_net(theta,x)[1]
    return w[0]+w[1]*r1+w[2]*r1**2+w[3]*r2+w[4]*r2**2

In [15]:
def square_loss(labels,predictions):
    loss=0
    for l,p in zip(labels,predictions):
        loss=loss+(l-p)**2
    loss=loss/len(labels)
    return loss

In [16]:
def cost(theta,w,features,labels):
    preds=[classical_quantum_net(theta,w,x) for x in features]
    return square_loss(labels,preds)

## Model training

In [17]:
w=np.zeros(5,requires_grad=True)

Using the Adam optimizer, we update the weights for 200 steps (this takes some time). More steps will lead to a better fit.

In [18]:
opt = AdamOptimizer(0.05, beta1=0.9, beta2=0.999)

In [19]:
start=now()
print(start)

2022-08-07 14:44:40.696400


In [20]:
epochs=200
for e in range(1,epochs+1):
    
    (theta,w,_,_),_cost=opt.step_and_cost(cost,theta,w,X,Y)

    if e==1 or e%10==0:
        print(f'Epoch: {e} | Cost: {_cost} | w: {w}')

Epoch: 1 | Cost: 0.6907793588973378 | w: [ 0.04999999 -0.04999993  0.04999973 -0.0499999   0.04999983]
Epoch: 10 | Cost: 0.1329426685835965 | w: [ 0.40978733 -0.41466322  0.39781239 -0.42665756  0.43384529]
Epoch: 20 | Cost: 0.08000577069457783 | w: [ 0.50392821 -0.40725699  0.30350214 -0.50388307  0.5022412 ]
Epoch: 30 | Cost: 0.06508315742801542 | w: [ 0.55119694 -0.40707994  0.24199637 -0.59361428  0.45815753]
Epoch: 40 | Cost: 0.05438892579953988 | w: [ 0.53923205 -0.41295714  0.1908594  -0.66040488  0.29432916]
Epoch: 50 | Cost: 0.04578805143718515 | w: [ 0.51531286 -0.45471438  0.17928896 -0.75168674  0.09528257]
Epoch: 60 | Cost: 0.0365385417386598 | w: [ 0.49292493 -0.50093943  0.186229   -0.87639567 -0.07241903]
Epoch: 70 | Cost: 0.02836434785280098 | w: [ 0.47542332 -0.55476581  0.19514115 -0.99059426 -0.27263165]
Epoch: 80 | Cost: 0.022346221649715874 | w: [ 0.47018543 -0.610666    0.19704911 -1.11765859 -0.45781145]
Epoch: 90 | Cost: 0.017604798035319898 | w: [ 0.47493371 -

In [21]:
end=now()
print(end)

2022-08-07 17:15:58.643547


In [22]:
print(end-start)

2:31:17.947147


## Training error

In [23]:
pred_train=[classical_quantum_net(theta,w,x) for x in X]

In [24]:
train_diff=np.abs(Y-pred_train)

In [25]:
np.max(train_diff)

tensor(0.27076842, requires_grad=True)

In [26]:
np.min(train_diff)

tensor(1.22874173e-05, requires_grad=True)

In [27]:
np.mean(train_diff)

tensor(0.03749004, requires_grad=True)

## Test error

In [28]:
test1,test2,test3=np.meshgrid(np.linspace(-0.95,0.95,30),np.linspace(-0.95,0.95,30),np.linspace(-0.95,0.95,30))
test1_flatten=test1.flatten()
test2_flatten=test2.flatten()
test3_flatten=test3.flatten()
Y_test=my_objective(test1_flatten,test2_flatten,test3_flatten)
pred_test=[classical_quantum_net(theta,w,x) for x in zip(test1_flatten,test2_flatten,test3_flatten)]

In [29]:
test_diff=np.abs(Y_test-pred_test)

In [30]:
np.max(test_diff)

tensor(0.37038266, requires_grad=True)

In [31]:
np.min(test_diff)

tensor(5.8766834e-08, requires_grad=True)

In [32]:
np.mean(test_diff)

tensor(0.04083191, requires_grad=True)

In [33]:
print(theta)

[[ 3.60274368  4.72870467  4.49417372]
 [ 6.80200921  5.83940347 -0.07342025]
 [ 6.12625035  0.63904894  0.04285385]
 [ 3.7176137  -0.12860071  3.19407229]]


## Randomly generate test dataset

In [34]:
number=27000
Test=np.random.uniform(-0.95,0.95,size=(number,3))

In [35]:
Test.shape

(27000, 3)

In [36]:
Y_Test=my_objective(Test[:,0],Test[:,1],Test[:,2])
print(Y_test.shape)

(27000,)


In [37]:
pred_Test=[classical_quantum_net(theta,w,x) for x in Test]

In [38]:
Test_diff=np.abs(Y_Test-pred_Test)

In [39]:
np.max(Test_diff)

tensor(0.3459091, requires_grad=True)

In [40]:
np.min(Test_diff)

tensor(1.41515283e-07, requires_grad=True)

In [41]:
np.mean(Test_diff)

tensor(0.03837082, requires_grad=True)